# Testing Notebook 02:

In [1]:
!pip install ..

from typing import List
import pandas as pd 
import numpy as np

from statdepth.depth import banddepth
from statdepth.depth.depth import _univariate_band_depth, _handle_depth_errors
from statdepth.depth._containment import _select_containment



Processing /Users/julian/Documents/Projects/statdepth
  Created wheel for statdepth: filename=statdepth-0.1.0-py2.py3-none-any.whl size=10908 sha256=a95732a011f7238c16370b83bde3235ac12a20031aa97a2d553ab95a87fef011
  Stored in directory: /private/var/folders/pd/jsjcl0fn7w57s5mfr34b20pm0000gn/T/pip-ephem-wheel-cache-jytxmhns/wheels/94/78/65/e7cb177a6619a78b187853c88271a8abc224aa673b05e4ed56
Successfully built statdepth
  Attempting uninstall: statdepth
    Found existing installation: statdepth 0.1.0
    Uninstalling statdepth-0.1.0:
      Successfully uninstalled statdepth-0.1.0


In [2]:
!where python

/Users/julian/opt/miniconda3/bin/python
/usr/bin/python


Generate a toy DataFrame for testing

In [3]:
t = [1,2,3,4,4,4,4.5,2,3,2,3]*2
df = pd.DataFrame()
for col in range(15):
    df[col] = t
    
for col in df:
    df[col] = df[col] * np.random.rand() * 2

df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.171552,0.842845,1.544763,1.356408,1.931888,1.942065,1.222123,1.913352,0.275942,1.590626,1.303740,0.971384,0.367003,0.409565,0.290045
1,0.343104,1.685690,3.089526,2.712817,3.863777,3.884130,2.444246,3.826704,0.551885,3.181251,2.607479,1.942768,0.734005,0.819130,0.580090
2,0.514657,2.528535,4.634289,4.069225,5.795665,5.826195,3.666370,5.740056,0.827827,4.771877,3.911219,2.914152,1.101008,1.228696,0.870135
3,0.686209,3.371380,6.179052,5.425634,7.727554,7.768261,4.888493,7.653408,1.103770,6.362503,5.214958,3.885536,1.468010,1.638261,1.160180
4,0.686209,3.371380,6.179052,5.425634,7.727554,7.768261,4.888493,7.653408,1.103770,6.362503,5.214958,3.885536,1.468010,1.638261,1.160180
5,0.686209,3.371380,6.179052,5.425634,7.727554,7.768261,4.888493,7.653408,1.103770,6.362503,5.214958,3.885536,1.468010,1.638261,1.160180
6,0.771985,3.792802,6.951433,6.103838,8.693498,8.739293,5.499554,8.610084,1.241741,7.157816,5.866828,4.371228,1.651512,1.843044,1.305202
7,0.343104,1.685690,3.089526,2.712817,3.863777,3.884130,2.444246,3.826704,0.551885,3.181251,2.607479,1.942768,0.734005,0.819130,0.580090
8,0.514657,2.528535,4.634289,4.069225,5.795665,5.826195,3.666370,5.740056,0.827827,4.771877,3.911219,2.914152,1.101008,1.228696,0.870135
9,0.343104,1.685690,3.089526,2.712817,3.863777,3.884130,2.444246,3.826704,0.551885,3.181251,2.607479,1.942768,0.734005,0.819130,0.580090


In [4]:
def samplebanddepth(data: List[pd.DataFrame], K: int, J=2, containment='r2', relax=False, deep_check=False):
    samples = []

    # Handle common errros
    _handle_depth_errors(data=data, J=J, containment=containment, relax=relax, deep_check=deep_check)

    cdef = _select_containment(containment=containment)
    
    # Univariate case
    if len(data) == 1:
        df = data[0]
        orig = df.copy()
        ss = df.shape[1] // K
        
        for i in range(K):
            depths = []
            t = df.sample(n=ss, axis=1)
            df = df.drop(t.columns, axis=1)

            for col in orig.columns:
                depths.append(_univariate_band_depth(data=t, curve=orig[col], relax=relax, containment=cdef, J=J))

            samples.append(depths)
        print('GOT HERE')
        print(samples)
        samples = pd.Series(index=df.columns, data=[np.mean(i) for i in samples])

    else:
        # Multivariate case: partition list of DataFrames randomly, compute band depth w.r.t those
        shuffled = data.copy()
        random.shuffle(shuffled)
        ss = len(data) // K 

        for _ in range(K):
            pass
        
    return samples

In [5]:
df = pd.DataFrame([
    [1, 2, 3, 6, 9, 8],
    [2, 4, 4, 7, 9, 8],
    [3, 5, 4, 6.5, 12, 10],
    [2, 6, 2, 6, 11, 10],
    [1, 2, 1, 7, 11, 9],
], columns=['f_0', 'f_1', 'f_2', 'f_3', 'f_4', 'f_5'], index=['x_0', 'x_1', 'x_2', 'x_3', 'x_4'])


In [39]:
np.array(samplebanddepth([df], K=2))

GOT HERE
[[0.6666666666666666, 1.0, 0.3333333333333333, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666], [0.0, 0.0, 0.6666666666666666, 1.0, 0.0, 0.6666666666666666]]


ValueError: Length of passed values is 2, index implies 0.

In [20]:
import statdepth

s=statdepth.depth.samplebanddepth([df], K=1)
pd.Series(index=df.columns, data=[np.mean(i) for i in s])

f_0    0.333333
f_1    0.533333
f_2    0.533333
f_3    0.733333
f_4    0.333333
f_5    0.600000
dtype: float64

In [40]:
statdepth.depth.banddepth([df]).sort_values()

f_0    0.333333
f_4    0.333333
f_1    0.533333
f_2    0.533333
f_5    0.600000
f_3    0.733333
dtype: float64